In [ ]:
#| default_exp server

In [ ]:
#| export
import sys

import zmq
import numpy as np

import whisper

In [ ]:
#| export
from asr_teach.audio import AudioBuffer
from asr_teach.utils import ZMQ_ARGS, PROTOCOL, ADDR, PORT

In [ ]:
#| export 

def recv_array(socket, flags=0, copy=True, track=False):
    '''Receives a numpy array over `socket`.
    
    Assumes a two-part message:
        First message has array data type and shape.
        Second message has the raw array bytes. 
        
    Returns the appropriately cast and reshaped numpy array. 
    '''
    md = socket.recv_json(flags=flags)
    msg = socket.recv(flags=flags, copy=copy, track=track)
    buf = memoryview(msg)
    arr = np.frombuffer(buf, dtype=md['dtype']).reshape(md['shape']).squeeze()
    return arr


def parse_results(result):
    '''Extracts helpful fields from the output of whisper's `transcribe`.
    
    Information parsed:
        - The transcribed text.  
        - Segment start and stop times in seconds.  
        - Log probability of the transcription.  
        - Probability that the segment had no speech.  
    '''
    parsed = []
    for seg in result['segments']:
        info = {'text': seg['text'],
                'beg': seg['start'],
                'end': seg['end'],
                'logprob': seg['avg_logprob'],
                'no_speech_prob': seg['no_speech_prob']}
        parsed.append(info)
    return parsed

In [ ]:
#| export

class WhisperServer:
    '''Runs a whisper model on audio streaming over a port.
    '''
    def __init__(self,
                 context = None,
                 buffer_size: int = 16_000,
                 model_name: str = 'base',
                 protocol: str = PROTOCOL,
                 addr: str = ADDR,
                 port: int = PORT):
        
        # create the socket to receive streaming audio
        context = context or zmq.Context()
        socket = context.socket(zmq.PULL)
        socket.connect(f'{protocol}://{addr}:{port}')
        self.socket = socket
        
        # initialize the audio buffer
        self.buffer = AudioBuffer(size=buffer_size)
        
        # load the whisper model
        self.model_name = model_name
        self.model = whisper.load_model(model_name)

        
    def accumulate(self, zmq_args=ZMQ_ARGS):
        '''Reads in data via `socket` and puts it into the audio buffer.
        '''
        arr = recv_array(self.socket, **zmq_args)
        self.buffer.accumulate(arr)

        
    def buffer_ready(self):
        '''Checks if the buffer is full with fresh data.
        '''
        return self.buffer.is_full

    
    def transcribe(self):
        '''Runs whisper trascription on the full audio buffer.
        '''
#         data = self.buffer.get_data()
        data = recv_array(self.socket)
#         data = np.array(data).astype(np.float32)
        audio = whisper.pad_or_trim(data)
        result = self.model.transcribe(audio)
        parsed = parse_results(result)
        print('\n'.join(seg['text'] for seg in parsed))
        self.reset_buffer()

        
    def reset_buffer(self):
        '''Prepares the buffer for the next audio window.'''
        self.buffer.reset()
        self.buffer.flush()

        
    def run(self):
        '''Main server function.
        
        TODO: catch interrupts (keyboard and signals)
        TODO: graceful shutdown
        '''
        while True:
            try:
#                 self.accumulate()
#                 if self.buffer_ready():
                self.transcribe()

            except KeyboardInterrupt:
                print('User keyboard interrupt.')
                print('Stopping server')
                break
                print('Server stopped')
            except Exception as e:
                print(f'Exception: {e}')
                raise
